<a href="https://colab.research.google.com/github/EhsaasN/LLM-learning/blob/main/finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install datasets

In [ ]:
from transformers import AutoTokenizer
from datasets import load_dataset

In [ ]:
template_tokenizer = AutoTokenizer.from_pretrained(
 "TinyLlama/TinyLlama-1.1B-Chat-v0.6"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
def format_prompt(example):
 """Format the prompt to using the <|user|> template TinyLLama is using"""
 # Format answers
 chat = example["messages"]
 prompt = template_tokenizer.apply_chat_template(chat, tokenize=False)
 return {"text": prompt}

In [ ]:
dataset = (
 load_dataset("HuggingFaceH4/ultrachat_200k", split="test_sft")
 .shuffle(seed=42)
 .select(range(3_000))
)
dataset = dataset.map(format_prompt)

In [ ]:
print(dataset["text"][2576])

<|user|>
Given the text: Knock, knock. Who’s there? Hike.
Can you continue the joke based on the given text material "Knock, knock. Who’s there? Hike"?</s>
<|assistant|>
Sure! Knock, knock. Who's there? Hike. Hike who? Hike up your pants, it's cold outside!</s>
<|user|>
Can you tell me another knock-knock joke based on the same text material "Knock, knock. Who's there? Hike"?</s>
<|assistant|>
Of course! Knock, knock. Who's there? Hike. Hike who? Hike your way over here and let's go for a walk!</s>



In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
%pip install bitsandbytes

In [ ]:
model_name = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"
# 4-bit quantization configuration - Q in QLoRA
bnb_config = BitsAndBytesConfig(
 load_in_4bit=True, # Use 4-bit precision model loading
 bnb_4bit_quant_type="nf4", # Quantization type
 bnb_4bit_compute_dtype="float16", # Compute dtype
 bnb_4bit_use_double_quant=True, # Apply nested quantization
)

In [ ]:
!pip install -U bitsandbytes
!pip install -U transformers accelerate

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
 model_name,
 device_map="auto",
 # Leave this out for regular SFT
 quantization_config=bnb_config,
)

model.safetensors:   0%|          | 0.00/4.40G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

In [ ]:
model.config.use_cache = False
model.config.pretraining_tp = 1
# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = "<PAD>"
tokenizer.padding_side = "left"

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
# Prepare LoRA Configuration
peft_config = LoraConfig(
 lora_alpha=32, # LoRA Scaling
 lora_dropout=0.1, # Dropout for LoRA Layers
 r=64, # Rank
 bias="none",
 task_type="CAUSAL_LM",
 target_modules= # Layers to target
 ["k_proj", "gate_proj", "v_proj", "up_proj", "q_proj", "o_proj",
"down_proj"]
)
# Prepare model for training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

In [ ]:
from transformers import TrainingArguments
output_dir = "./results"
# Training arguments
training_arguments = TrainingArguments(
 output_dir=output_dir,
 per_device_train_batch_size=2,
 gradient_accumulation_steps=4,
 optim="paged_adamw_32bit",
 learning_rate=2e-4,
 lr_scheduler_type="cosine",
 num_train_epochs=1,
 logging_steps=10,
 fp16=True,
 gradient_checkpointing=True
)

In [ ]:
%pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 8.3 MB/s eta 0:00:00


In [ ]:
!pip install -U trl


In [ ]:
%pip install -U transformers

In [ ]:
import transformers
print(transformers.__version__)

4.51.1


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

# Define training arguments
training_arguments = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    #evaluation_strategy="no",  # or "steps" if using a val set
    save_total_limit=1,
    remove_unused_columns=False,
    fp16=True,  # optional, for faster training
)
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    #dataset_text_field="text",        # column name in your dataset
    tokenizer=tokenizer,
    args=training_arguments,
    max_seq_length=512,
    peft_config=peft_config,          # QLoRA config (optional)
)

# Start training
trainer.train()

# Save adapter weights (QLoRA)
trainer.model.save_pretrained("TinyLlama-1.1B-qlora")

TypeError: SFTTrainer.__init__() got an unexpected keyword argument 'tokenizer'